# Coastal change and impact analysis

* **Products used:** [DE Africa Coastlines](), [High Resolution Population Density]()


## Background

Coastal changes have a larget impact on human settlements. 

## Description

This notebook will demonstrate how to load relevant datasets and analyse the impact of coastal changes on human settlements.
Topics covered include:

1. Loading DE Africa Coastlines rates of change statistics data using the `get_coastlines` function.
2. Loading High Resolution Population Density.
3. Combine


## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
import geopandas as gpd
import rioxarray as rxr
import numpy as np
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.coastal import get_coastlines
from deafrica_tools.plotting import display_map
from datacube.utils.geometry import Geometry


# set s3 region for HRSL data access
import os
os.environ['AWS_DEFAULT_REGION']="us-east-1"
os.environ['AWS_S3_ENDPOINT']="s3.us-east-1.amazonaws.com"

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### Analysis parameters

This section defines the analysis parameters, including:

* `central_lat, central_lon, buffer`: center lat/lon and analysis window size for the area of interest

The default location is an area on Nyali Beach along the Kenyan Coast.

In [2]:
african_countries  = gpd.read_file("../Supplementary_data/MGCI/african_countries.geojson")

In [3]:
np.unique(african_countries["COUNTRY"])

array(['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso',
       'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic',
       'Chad', 'Comoros', 'Congo-Brazzaville', 'Cote d`Ivoire',
       'Democratic Republic of Congo', 'Djibouti', 'Egypt',
       'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia',
       'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia',
       'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Morocco',
       'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda',
       'Sao Tome and Principe', 'Senegal', 'Sierra Leone', 'Somalia',
       'South Africa', 'Sudan', 'Swaziland', 'Tanzania', 'Togo',
       'Tunisia', 'Uganda', 'Western Sahara', 'Zambia', 'Zimbabwe'],
      dtype=object)

In [4]:
country = "Liberia"

In [5]:
idx = african_countries[african_countries['COUNTRY'] == country].index[0]
country_geom = Geometry(geom=african_countries.iloc[idx].geometry, crs=african_countries.crs)

In [6]:
bbox = list(country_geom.boundingbox)

## View the selected location

The next cell will display the selected area on an interactive map. Feel free to zoom in and out to get a better understanding of the area you'll be analysing. Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [7]:
display_map(x=(bbox[0], bbox[2]), y=(bbox[1], bbox[3]))

## Loading Population data

In [8]:
ds = rxr.open_rasterio('s3://dataforgood-fb-data/hrsl-cogs/hrsl_general/hrsl_general-latest.vrt', chunks ={'x': 1000, 'y': 1000})

### Population within a buffer of 2021 shoreline

In [9]:
buffer = 50 #meter

In [10]:
# Load DE Africa Coastlines annual shorelines data. 
deacl_shorelines_gdf = get_coastlines(bbox=bbox, 
                                      layer='shorelines')

/usr/local/lib/python3.8/dist-packages/geopandas/tools/clip.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [11]:
buffered = deacl_shorelines_gdf.copy()

In [12]:
buffered['geometry'] = deacl_shorelines_gdf.to_crs('6933').buffer(buffer).to_crs('4326')

In [13]:
coast_bbox = buffered.unary_union.bounds
coast_bbox

(-11.492598206286239, 4.354116928570664, -7.364591791808162, 6.92300308467692)

In [ ]:
pop_raster = ds.isel(band=0).sel(x=slice(coast_bbox[0],coast_bbox[2]), y = slice(coast_bbox[3], coast_bbox[1])).compute()
#youth_raster = ds_youth.isel(band=0).sel(x=slice(coastal_bbox[0],coastal_bbox[2]), y = slice(coastal_bbox[3],coastal_bbox[1]))

pop_raster=pop_raster.rename({'x':'longitude','y':'latitude'})
#youth_raster=youth_raster.rename({'x':'longitude','y':'latitude'})


country_mask = xr_rasterize(gdf=african_countries[african_countries['COUNTRY'] == country],
                                  da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)

In [ ]:
coastal_pop = {}
for year in range(2000,2021):
    coastal_area = buffered[buffered.year==str(year)]
    coastal_raster = xr_rasterize(gdf=coastal_area,
                                  da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)
    
    #fraction of coastal population
    coastal_pop[year] = (coastal_raster*pop_raster*country_mask).sum().values
    print(year, coastal_pop[year]) 

In [ ]:
import pandas as pd
import numpy as np

# convert to million
pd.DataFrame(np.array(list(coastal_pop.values()))/1e6, 
             columns=[f'General'], 
             index=[str(x) for x in coastal_pop.keys()]
            ).plot(ylabel=f'Population within {buffer}m from coast (million)',xlabel='Year')